In [18]:
import requests
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
import ssl
from utils import get_access_token

# Search for available Sentinel-2 images
def search_available_dates(lat, lon, target_date, max_days=30):
    access_token = get_access_token()
    
    target_date = datetime.strptime(target_date, "%Y-%m-%d")
    start_date = (target_date - timedelta(days=max_days)).strftime("%Y-%m-%d")
    end_date = (target_date + timedelta(days=max_days)).strftime("%Y-%m-%d")
    
    url = "https://services.sentinel-hub.com/api/v1/catalog/search"
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "datetime": f"{start_date}T00:00:00Z/{end_date}T23:59:59Z",
        "collections": ["sentinel-2-l2a"],
        "limit": 100,
        "intersects": {
            "type": "Point",
            "coordinates": [lon, lat]
        }
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code != 200:
        print("Error:", response.json())
        return None
    
    data = response.json().get("features", [])
    
    if not data:
        print("No available data found.")
        return None
    
    # Extract dates from the results
    available_dates = sorted([datetime.strptime(item["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ") for item in data])
    return available_dates



In [17]:
def get_cloud_coverage(lat, lon, date_list):
    access_token = get_access_token()
    
    url = "https://services.sentinel-hub.com/api/v1/catalog/search"
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    cloud_coverage_results = {}
    
    for date in date_list:
        start_date = datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
        end_date = datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")
        
        payload = {
            "datetime": f"{start_date}/{end_date}",
            "collections": ["sentinel-2-l2a"],
            "limit": 1,  # Get only the first available image for that date
            "intersects": {
                "type": "Point",
                "coordinates": [lon, lat]
            }
        }
        
        response = requests.post(url, json=payload, headers=headers)
        
        if response.status_code != 200:
            print(f"Error fetching data for {date}: {response.json()}")
            cloud_coverage_results[date] = None
            continue
        
        data = response.json().get("features", [])
        
        if not data:
            cloud_coverage_results[date] = None
        else:
            cloud_coverage = data[0]["properties"].get("eo:cloud_cover", "Unknown")
            cloud_coverage_results[date] = cloud_coverage
    
    return cloud_coverage_results

In [19]:
latitude = 25.286611
longitude = 55.462611
target_date = "2025-02-04"  # YYYY-MM-DD format

available_dates = search_available_dates(latitude, longitude, target_date)
available_dates = [i.strftime("%Y-%m-%d") for i in available_dates]
available_dates_cloud_coverage = get_cloud_coverage(latitude, longitude, available_dates)
available_dates_cloud_coverage

{'2025-01-07': 13.45,
 '2025-01-12': 0.0,
 '2025-01-17': 57.11,
 '2025-01-22': 0.23,
 '2025-01-27': 0.83,
 '2025-02-01': 0.04,
 '2025-02-06': 18.9,
 '2025-02-11': 11.02,
 '2025-02-16': 68.75,
 '2025-02-21': 33.53,
 '2025-02-26': 7.18,
 '2025-03-03': 0.29}